In [1]:
import numpy as np
import shorttext
import pandas as pd

Using TensorFlow backend.


# Data

In [2]:
# source: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
imdbdata = pd.read_csv('/data/hok/testdata/imdb/IMDB Dataset.csv')

In [3]:
imdbdata.groupby('sentiment').count()

,review
sentiment,
negative,25000
positive,25000


In [4]:
imdbdata.iloc[:100,].groupby('sentiment').count()

,review
sentiment,
negative,58
positive,42


In [5]:
imdbdata = imdbdata.iloc[:100,]
imdbdata

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
95,Daniel Day-Lewis is the most versatile actor a...,positive
96,My guess would be this was originally going to...,negative
97,"Well, I like to watch bad horror B-Movies, cau...",negative
98,"This IS the worst movie I have ever seen, as w...",negative


# Data Conversion Pipeline

In [6]:
import re

In [7]:
remove_htmltags = lambda s: re.sub(r'<(/)*(\w)+([\s|/])*>', '', s)
remove_specialchars = lambda s: re.sub(r'[^A-Za-z\s]', ' ', s)
remove_consecutivespaces = lambda s: re.sub(r'\s+', ' ', s)
convert_tolowercase = lambda s: s.lower()
strip_string = lambda s: s.strip()

In [8]:
pipeline = [remove_htmltags, remove_specialchars, remove_consecutivespaces, convert_tolowercase, strip_string]

In [9]:
preprocess = shorttext.utils.text_preprocessor(pipeline)

# Word2Vec Model

In [10]:
wmodel = shorttext.utils.load_word2vec_model('/data/hok/langmodel/word2vec/GoogleNews-vectors-negative300.bin')

In [11]:
from collections import defaultdict

trainclassdict = defaultdict(lambda : [])
for review, sentiment in zip(imdbdata['review'], imdbdata['sentiment']):
    trainclassdict[sentiment] += [preprocess(review)]
    
trainclassdict = dict(trainclassdict)

In [12]:
kmodel = shorttext.classifiers.frameworks.CNNWordEmbed(2, vecsize=wmodel.vector_size)

In [13]:
classifier = shorttext.classifiers.VarNNEmbeddedVecClassifier(wmodel)

In [14]:
classifier.train(trainclassdict, kmodel)

Epoch 1/10
100/100 [==============================] - 0s 3ms/step - loss: 0.6904
Epoch 2/10
100/100 [==============================] - 0s 989us/step - loss: 0.4497
Epoch 3/10
100/100 [==============================] - 0s 999us/step - loss: 0.3150
Epoch 4/10
100/100 [==============================] - 0s 989us/step - loss: 0.2589
Epoch 5/10
100/100 [==============================] - 0s 963us/step - loss: 0.1820
Epoch 6/10
100/100 [==============================] - 0s 979us/step - loss: 0.1499
Epoch 7/10
100/100 [==============================] - 0s 993us/step - loss: 0.1111
Epoch 8/10
100/100 [==============================] - 0s 1ms/step - loss: 0.0905
Epoch 9/10
100/100 [==============================] - 0s 977us/step - loss: 0.0626
Epoch 10/10
100/100 [==============================] - 0s 969us/step - loss: 0.0494


In [15]:
classifier.score('this movie is bad')

{'positive': 0.17456181, 'negative': 0.8254382}

In [16]:
classifier.score('this movie is fabulous')

{'positive': 0.69592625, 'negative': 0.30407372}

In [17]:
classifier.save_compact_model('imdb_cnn.bin')